<a href="https://colab.research.google.com/github/JuanM-GG/sistemas-dinamicos-R/blob/main/fed_bach_3_enero_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Título: fed-bach bioreactor 

Nombre: JM

Fecha: enero 2021

Cargar librerías

In [ ]:
install.packages("deSolve")
library(deSolve)

In [ ]:
library(ggplot2)
source("Grind.R")

# establecer modelo 
model <- function(times, state, parms) {
        with(as.list(c(state,parms)), {
                
                # velocidad de crecimiento de biomasa
                rg = mu*s/(ks + s + ki*s^2)*x
                
                # velocidad de consumo de sustrato
                rs = (1/y)*rg
                
                # balance de material global 
                dVdt = Fin 
                
                # balance de materia para sustrato
                dsdt = Fin/V*(sf - s) - rs
                
                # balance de materia para biomasa
                dxdt = - Fin/V*x + rg
                
                return(list(c(dVdt, dsdt, dxdt)))
        })
}

p <- c(mu = 1.2, ks = 280, y = 0.2, sf = 40, Fin = 0, ki = 1)

s <- c(V = 10.0, s = 10, x = 0.05)

times <- seq(0,100,len=100)

out <- ode(y = s,times = times,func = model,parms = p,method = "rk4")

plot(out)

m <- nrow(out)

x_end <- out[m,2]*out[m,4]

x_end
# a pesar que las ecuaciones para biomasa y sustrato son exactamente iguales que en el caso del
# reactor continuo, no se presenta equilibrio porque el volumen siempre está aumentando 

# efecto de Fin
n <- length(times)
Fin <- rep(0, n)
Fin[50:75] <- 5
Fin[76:n] <- 0.5

# vectores para guardar los resultados 
V0 <- 10
V <- rep(V0, n)
s0 <- 10
su <- rep(s0,n)
x0 <- 0.05
x <- rep(x0,n)

# resolver EDO para los diferentes valores de Fin
for (i in 1:(n-1)) {
        
        tspan <- c(times[i], times[i+1])
        
        # actualizar Fin 
        p["Fin"] <- Fin[i+1]
        
        # resolver EDOs
        out <- ode(y = s,
                   times = tspan,
                   func = model,
                   parms = p,
                   method = "rk4")
        m <- nrow(out)
        
        # guardar resultados 
        V[i+1] <- out[m,2]
        su[i+1] <- out[m,3] 
        x[i+1] <- out[m,4]
        
        # actualizar condiciones iniciales
        s <- out[m,c(2,3,4)]
        
}

par(mfrow = c(1,1))
plot(times, V, type = "l")
plot(times, su, type = "l")
plot(times, x, type = "l")

# función para obtener la masa final de biomasa 
biomass1 <- function(times, state, parms) {
        
        out <- ode(y = state,times = times,func = model,parms = parms,method = "rk4")
        m <- nrow(out)
        
        ms_end <- out[m,2]*out[m,3]
        mx_end <- out[m,2]*out[m,4]
        V_end <- out[m,2]
        s_end <- out[m,3]
        x_end <- out[m,4]
        
        return(list(ms_end = ms_end, mx_end = mx_end, V_end = V_end, s_end = s_end, x_end = x_end))
}

# función para obtener la biomasa final para diferentes valores de Fin 
biomass2 <- function(times, state, parms, Fin) {
        
        n <- length(Fin)
        ms_end <- numeric(length = n)
        mx_end <- numeric(length = n)
        V_end <- numeric(length = n)
        s_end <- numeric(length = n)
        x_end <- numeric(length = n)
        for (i in 1:n) {
                
                parms["Fin"] <- Fin[i]
                
                out <- biomass1(times = times,state = state,parms = parms)
                
                ms_end[i] <- out$ms_end
                mx_end[i] <- out$mx_end
                V_end[i] <- out$V_end
                s_end[i] <- out$s_end
                x_end[i] <- out$x_end
                
        }
        
        return(list(ms_end = ms_end, mx_end = mx_end, V_end = V_end, s_end = s_end, x_end = x_end))
}

# si ki = 0 no hay efecto de inhibición y la biomasa siempre aumenta
# como si los  bichos pudieran comer todo el sustrato sin problema :V
# con ki <- 0.4 tengo un óptimo de Fin = 2 aprox

p["ki"] <- 0.6
Fin <- seq(0,5,len=100)
out <- biomass2(times = times,state = s,parms = p,Fin = Fin)
ms_end <- out$ms_end
mx_end <- out$mx_end
V_end <- out$V_end
s_end <- out$s_end
x_end <- out$x_end


plot(Fin, ms_end, type = "l")
plot(Fin, mx_end, type = "l")
plot(Fin, V_end, type = "l")
plot(Fin, s_end, type = "l")
plot(Fin, x_end, type = "l")

Fin <- seq(0,10,len=100)
p
p["sf"] <- 50
j <- 1
for (ki in c(0.3,0.35,0.4,0.45,0.5,0.6)) {
        
        p["ki"] <- ki
        out <- biomass2(times = times, state = s, parms = p, Fin = Fin)
        mx_end <- out$mx_end
        
        if (j == 1) {
                
                plot(Fin, mx_end, type = "l", col = j, ylim = c(400,4000))
        }
        else {
                lines(Fin, mx_end, type = "l", col = j)
        }
        j = j + 1
}
p["sf"] <- 40

# función para obtener la biomasa final para diferentes valores de Fin 
biomass3 <- function(times, state, parms, Fin, ki) {
        
        n <- length(ki)
        
        # lista para guardar los vectores que regresa biomass2()
        mx_end <- list(length = n)
        
        for (i in 1:n) {
                
                parms["ki"] <- ki[i]
                
                out <- biomass2(times = times,state = state,parms = parms,Fin = Fin)
                
                mx_end[[i]] <- out$mx_end #<- esto es un vector
                
        }
        
        return(mx_end)
}

ki <- c(0.3,0.35,0.4,0.45,0.5,0.6)


for (sf in seq(0,50,5)) {
        
        p["sf"] <- sf
        
        out <- biomass3(times = times,state = s,parms = p,Fin = Fin,ki = ki)
        
        for (i in 1:length(out)) {
                
                if (i == 1) {
                        
                        plot(Fin, out[[i]],ylab = "mx_end", main = paste0("sf = ", sf),
                        type = "l", col = i, ylim = c(400,4000))
                }
                
                else {
                        lines(Fin, out[[i]], type = "l", col = i)
                }
        }
        
}

Declarar la función